In [17]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [18]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

In [19]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [20]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [21]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [22]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [23]:
## 
n='Input_A1_020' # 改名稱
##
dire= ['./Data_Science_2020_imdb/BayesianRidge_Pre/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_A1_020
./Data_Science_2020_imdb/BayesianRidge_Pre/
feature number: 2
train_rmse: 0.5928217486273215
test_rmse: 0.646585647097155
test_r2: 0.370698679108309
feature number: 3
train_rmse: 0.584472027661452
test_rmse: 0.6511332225562532
test_r2: 0.3648513553125811
feature number: 4
train_rmse: 0.5831209556536728
test_rmse: 0.6568360159671985
test_r2: 0.3545339642275791
feature number: 5
train_rmse: 0.5824025653306407
test_rmse: 0.6594662460327358
test_r2: 0.34934478174008166
feature number: 6
train_rmse: 0.5820069840234381
test_rmse: 0.659608719896304
test_r2: 0.34914455883802875
feature number: 7
train_rmse: 0.5817708592190032
test_rmse: 0.6596553345534402
test_r2: 0.34894643246936924
feature number: 8
train_rmse: 0.5807490080212043
test_rmse: 0.6589953026259925
test_r2: 0.3504795277327843
feature number: 9
train_rmse: 0.5740197085483204
test_rmse: 0.6482825019788353
test_r2: 0.3703332570152188
feature number: 10
train_rmse: 0.5683738583407798
test_rmse: 0.6415633131733706
test_r2:

train_rmse: 0.5480228052584674
test_rmse: 0.6316929867784972
test_r2: 0.39996909500095607
feature number: 17
train_rmse: 0.5473087402500948
test_rmse: 0.6346172666740557
test_r2: 0.39429350015847897
feature number: 18
train_rmse: 0.5463074682467943
test_rmse: 0.6409081063338103
test_r2: 0.38229308352614155
feature number: 19
train_rmse: 0.5459588158279103
test_rmse: 0.6423494494293135
test_r2: 0.3794655468607739
feature number: 20
train_rmse: 0.5419630951882063
test_rmse: 0.6448431567128531
test_r2: 0.37400355414929487
Test_rmse_min = 1.fea_num: 14  2.rmse: 0.6306883477951887
Test_r2_max = 1. fea_num: 14  2.r2: 0.4020873162527403
./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.5928217486273215
test_rmse: 0.646585647097155
test_r2: 0.370698679108309
feature number: 3
train_rmse: 0.584472027661452
test_rmse: 0.6511332225562532
test_r2: 0.3648513553125811
feature number: 4
train_rmse: 0.5831209556536728
test_rmse: 0.6568360159671985
test_r2: 0.3545339642

train_rmse: 0.5683738583407798
test_rmse: 0.6415633131733706
test_r2: 0.38259426691083415
feature number: 11
train_rmse: 0.568269735931958
test_rmse: 0.6422544269354089
test_r2: 0.381295062391624
feature number: 12
train_rmse: 0.5611450167474727
test_rmse: 0.6384197324269967
test_r2: 0.3884534926582468
feature number: 13
train_rmse: 0.5524037506036045
test_rmse: 0.6330532791206039
test_r2: 0.3970024176123866
feature number: 14
train_rmse: 0.5496229917954911
test_rmse: 0.6306442436576548
test_r2: 0.4021177785473225
feature number: 15
train_rmse: 0.5489977944588103
test_rmse: 0.6309090424928407
test_r2: 0.40156610938717546
feature number: 16
train_rmse: 0.548011847249071
test_rmse: 0.6315559262749826
test_r2: 0.4001704584213247
feature number: 17
train_rmse: 0.5472923990757493
test_rmse: 0.6345618407486623
test_r2: 0.3943362121363598
feature number: 18
train_rmse: 0.5462939948416475
test_rmse: 0.6408738846676534
test_r2: 0.3822782628133728
feature number: 19
train_rmse: 0.545943396098935

# 可以修改第一層的神經元數量

In [24]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(12, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [25]:
fea_num=[2,10,11,12,13,14] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/BayesianRidge_Pre/Input_A1_020.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 2
test_rmse_ave: 0.5972190723947649
[0.4606124816499314, 0.7131329652705556, 0.7806782245020607, 0.4801149642252585, 0.6826257798795554, 0.7033701007318881, 0.5786206814036482, 0.6916154574309986, 0.47017671711065506, 0.4112433517430976]


test_r2_ave: 0.40526985996896175
[0.6018478008922414, 0.25894995925507913, 0.46898184721539615, 0.2992581034808802, 0.4690755168116757, 0.31988729034156416, 0.5431483908865566, 0.14786853946888812, 0.47240667811170023, 0.4712744732256361]


feature number: 10
test_rmse_ave: 0.6619641596349484
[0.5266951712587571, 0.6719712123032245, 0.8852511119439858, 0.5255191968583125, 0.6266841226737925, 0.7251214027245759, 0.7782996996920506, 0.7541103365969947, 0.502173921651796, 0.6238154206459938]


test_r2_ave: 0.24706871714169537
[0.47940932119931645, 0.34202734299301374, 0.3171926332409799, 0.16045347194036919, 0.5525291333125986, 0.27717272373152757, 0.17342744743128724, -0.013087612167146156, 0.39815402217535034, -0.21659131244034313]


f

In [28]:
fea_num=[2,10,11,12,13,14] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/Input_A1_020.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 2
test_rmse_ave: 0.5948177926889884
[0.4429453904563818, 0.7100324109345172, 0.7658877043610978, 0.48559092744770166, 0.6326533004321074, 0.7042136625631474, 0.569435347206717, 0.7561239864577428, 0.45521707380916904, 0.4260781232213031]


test_r2_ave: 0.4008417670198144
[0.6318048298347121, 0.26537981495127594, 0.4889122980031607, 0.28318228834604176, 0.5439641996636897, 0.31825497427978033, 0.5575378775089148, -0.01850519437512954, 0.505445524190572, 0.432441057795126]


feature number: 10
test_rmse_ave: 0.6908171584400392
[0.5695141662226608, 0.732197449366075, 0.8972565447822839, 0.6835606580302223, 0.6526187268094562, 0.7505780758899988, 0.8300251093736545, 0.6793542498896252, 0.500419884446305, 0.6126467195901119]


test_r2_ave: 0.16951460874200683
[0.39132316062270367, 0.21879874263009957, 0.2985471148324659, -0.4204361549013167, 0.5147266463660591, 0.22552958196610862, 0.05990938903374332, 0.17781450497284335, 0.40235104053977877, -0.17341793864241728]


feature

In [29]:
fea_num=[2,10,11,12,13,14] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/Input_A1_020.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 2
test_rmse_ave: 0.5958822445206805
[0.4460345117257796, 0.7171778479275174, 0.7814767921605017, 0.4886217330856006, 0.6432843091488829, 0.7070379567728063, 0.5618088602119222, 0.7238208349632486, 0.4634513994982612, 0.4261081997122843]


test_r2_ave: 0.40162812924247443
[0.6266513021356663, 0.2505196625609437, 0.4678949184522916, 0.2742063583808185, 0.5285091183379941, 0.31277564407423963, 0.5693103628006582, 0.06666107719862013, 0.48739192051885993, 0.43236092796465164]


feature number: 10
test_rmse_ave: 0.7011557321347792
[0.5747787355813241, 0.7197476429019153, 0.8897195988587636, 0.5801594860312784, 0.6832634840195501, 0.8478945325960034, 0.852618858969218, 0.6991055580985146, 0.5756246949344727, 0.5886447293567526]


test_r2_ave: 0.16552882960199722
[0.3800179731151545, 0.24513895690881504, 0.31028201475232364, -0.02320439052314538, 0.46808308172627755, 0.011681888177065947, 0.008033231743688307, 0.12931166244775516, 0.2092196193187501, -0.0832757416467127]


fea

In [30]:
fea_num=[2,10,11,12,13,14] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/Input_A1_020.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 2
test_rmse_ave: 0.5957517240180051
[0.44720470839158605, 0.7197335504836408, 0.7888553737124913, 0.4623521532279778, 0.6288082100644273, 0.7031489432948118, 0.5778018864768547, 0.7470075761906172, 0.44791517296369954, 0.4346896653739441]


test_r2_ave: 0.40284109886699077
[0.6246897296809617, 0.24516851647325621, 0.45779937634535783, 0.3501496327125032, 0.5494906711873531, 0.3203149125477377, 0.5444404398015564, 0.005906508382685249, 0.5211840601851665, 0.4092671413533294]


feature number: 10
test_rmse_ave: 0.7193453760357322
[0.6296979088537159, 0.7826473411447193, 0.9269168182109719, 0.6927070777846613, 0.657327242821146, 0.7746010962799968, 0.8372278356370773, 0.7680812781841279, 0.5168960346075016, 0.6073511268334044]


test_r2_ave: 0.10405590578335602
[0.2558813039059561, 0.1074372900968138, 0.2514052784432941, -0.4587029117533077, 0.5076990822569186, 0.17516078704940974, 0.04352290960002103, -0.05097304205747499, 0.3623484024597323, -0.15322004216780272]


featu

In [31]:
fea_num=[2,10,11,12,13,14] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/Input_A1_020.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 2
test_rmse_ave: 0.591366649404254
[0.4613786439653675, 0.6932014883592373, 0.779267940923617, 0.4588225375603578, 0.6147297731803522, 0.699943811305315, 0.5618252282802993, 0.7714614103843712, 0.4586913031315218, 0.41434435695210037]


test_r2_ave: 0.4097364265320172
[0.6005221619850387, 0.2997945555410729, 0.4708986670558427, 0.36003373009181516, 0.5694378221658876, 0.3264971457800657, 0.5692852665068, -0.06024353098124213, 0.4978678105280473, 0.463270636646844]


feature number: 10
test_rmse_ave: 0.6639485199667067
[0.553207483908975, 0.6666579117566448, 0.8523691563795027, 0.6218100835593231, 0.6891341565631529, 0.7418588626002384, 0.6694766356532318, 0.8090016507218097, 0.4327044610756495, 0.6032647974485388]


test_r2_ave: 0.23098470677821767
[0.425680175433957, 0.3523914326738413, 0.3669752530955649, -0.17539305595695476, 0.458903239836859, 0.24341859479791594, 0.3884131990011698, -0.1659395465758866, 0.5531520225714914, -0.13775424709578132]


feature number: 11

In [32]:
fea_num=[2,10,11,12,13,14] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/Input_A1_020.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 2
test_rmse_ave: 0.5878169980514615
[0.4431062618980416, 0.7046872266680739, 0.767558688233681, 0.46388205001102817, 0.6328922245137099, 0.7053913461222412, 0.5618506209511033, 0.7340709001491589, 0.46116569637900096, 0.40356496558857674]


test_r2_ave: 0.419260605560018
[0.6315373348105515, 0.2763987487451055, 0.4866797228211208, 0.3458418821745768, 0.543619687088156, 0.31597285084350424, 0.5692463318201902, 0.040039785831935126, 0.4924357313654085, 0.49083398009963086]


feature number: 10
test_rmse_ave: 0.6755701623894386
[0.46463205057490437, 0.7395287447145995, 0.8747962677769493, 0.6879481580401055, 0.6328455862749396, 0.7277986356247361, 0.7210159887001283, 0.810449942155796, 0.529794367647831, 0.5668918823843954]


test_r2_ave: 0.1953894853354694
[0.5948684710806157, 0.20307650899274665, 0.3332253514343043, -0.438729087479941, 0.5436869465327654, 0.2718253449131688, 0.2906230953628508, -0.17011786131169315, 0.3301281544183219, -0.00469207058844523]


feature num

In [33]:
fea_num=[2,10,11,12,13,14] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/Input_A1_020.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 2
test_rmse_ave: 0.5902059258268191
[0.44007956804924375, 0.7026613120574554, 0.7615078291624578, 0.466005138523491, 0.6338504136559624, 0.7016069503551402, 0.5660274561756261, 0.7341881940508824, 0.46087677223868917, 0.43525562399924195]


test_r2_ave: 0.41205684137226883
[0.6365538057521478, 0.2805533496688317, 0.49474108906895364, 0.3398402971322174, 0.5422367353773931, 0.32329271808146265, 0.5628180227080835, 0.03973298583558682, 0.4930715186026591, 0.4077278914953524]


feature number: 10
test_rmse_ave: 0.685055939690013
[0.5467881074455955, 0.688057925069821, 0.8750892481465399, 0.6576249190942178, 0.6463215691822812, 0.79323288351431, 0.7732357022256116, 0.7201685180213071, 0.512471786917038, 0.6375687372834088]


test_r2_ave: 0.17888095543652716
[0.4389315675255895, 0.310147070080416, 0.3327786540646891, -0.3146923093665408, 0.5240463292652173, 0.13500319408522599, 0.1841486233775813, 0.0760563558754973, 0.37321733947091573, -0.27082727001331985]


feature numbe

In [34]:
fea_num=[2,10,11,12,13,14] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/Input_A1_020.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 2
test_rmse_ave: 0.58732751874386
[0.435800496315159, 0.7043197931689743, 0.7756431106768884, 0.46088800663174295, 0.6241145817265631, 0.700285802312532, 0.5720420862870402, 0.7324672931569971, 0.4575051070307346, 0.4102089101319689]


test_r2_ave: 0.42049383688830994
[0.6435873127339244, 0.27715314307533934, 0.47580953730335673, 0.3542589232623591, 0.5561910659168037, 0.32583884096876015, 0.5534776333788392, 0.04422934701012571, 0.5004615247838138, 0.4739310404497775]


feature number: 10
test_rmse_ave: 0.6552284050086145
[0.48446834908378994, 0.6375824401108203, 0.8863433576631689, 0.5226377207769388, 0.6517097780718001, 0.6873368467398181, 0.6893884911087312, 0.7415511323155044, 0.4829251691504446, 0.7683407650651278]


test_r2_ave: 0.22886108788422943
[0.5595379123382833, 0.4076488918283987, 0.31550666300517105, 0.1696348722599469, 0.5160774547823304, 0.35054010655874224, 0.35149203891685576, 0.020375986846626337, 0.44340829156640926, -0.8456113392604692]


feature 